In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio



# mapping to the modules that make the app
sys.path.insert(0, "../..")
# sys.path.insert(0, "..")


In [2]:
from app.objects import structures

from notebooks.helpers import test_queries

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import app.models as models

c = models.CosmosdbClient()

executing local windows deployment


In [14]:
buildings = yaml.safe_load(
    open(os.path.join(os.getenv("abspath"), "app", "configurations", "buildings.yaml"))
)
buildings["explanation"]

[{'type': "all buildings have a 'type'/'label' of building"},
 {'name': "name_with_underscore. UI can remove the '_' at display time."},
 {'augments_resource': 'OPTIONAL augments a specific resource, creates it if it does not exist'},
 {'owned_by': 'type of object that can own/maintain this building, assuming a relationhsip to that object'},
 {'faction_augments': 'if owned by a pop in the faction,  raises the stat for all pops in factions'},
 {'effort': 'time in universal time units (UTU) that the action will take to complete'},
 {'requires_attr': 'required attributes of the agent. Determines if the agent can do the action.'},
 {'render_type': "guides the rendering in buildings.js. Available types: 'block'"},
 {'renews_faction_resource': 'augments (or diminishes) a resource in the faction. Creates if it does not exist.'},
 {'renews_location_resource': 'augments (or diminishes) a resource in the location. Creates if it does not exist.'},
 {'each_population_augments_once': 'augments a st

# Creating a new Building

In [5]:
username = "BillmanLocal2"
pop = test_queries.get_random_pop(c,username)
pop


ValueError: a must be greater than 0 unless no samples are taken

In [15]:
buildings_list = list(buildings["buildings"].keys())
buildings_list

['farm',
 'hospital',
 'solar_panel',
 'forrestry',
 'infrastructure',
 'university',
 'observatory',
 'prospector',
 'oil_well']

Should be able to search the building type by name

In [17]:
b = buildings["buildings"]['farm']
b

{'name': 'Farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'faction_augments': {'wealth': -1},
 'renews_faction_resource': {'granulated_hydrocarbons': 10},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block'}

In [19]:
building = structures.Building(pop,b)
building

NameError: name 'pop' is not defined

In [18]:
building.get_data()

NameError: name 'building' is not defined

In [10]:
building.get_owned_by()

{'node1': '8330456965074', 'node2': '1992522165129', 'label': 'owns'}

In [11]:
data = {'nodes':[building.get_data()],'edges':[building.get_owned_by()]}
c.upload_data(userguid,data)

In [12]:
building_query = f"""
g.V().has('objid','{pop['objid']}').as('pop')
    .in('owns').as('building')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('objid','name','changes','augments_resource','planet_requirements','description','render_type'))
"""

c.run_query(building_query)
c.res

[{'labels': [['pop'], ['building']],
  'objects': [{'objid': ['1992522165129'], 'name': ['Waquimac Lamdonzobat']},
   {'objid': ['8330456965074'],
    'name': ['farmland'],
    'changes': ['{wealth: -0.01}'],
    'augments_resource': ['{vegitable_foodstuff: 10}'],
    'planet_requirements': ['{isHabitable: true}'],
    'description': ['Generates organic foodstuffs, populations will consume food before consuming natural resources'],
    'render_type': ['block']}]}]